# Annotation Tutorial

**NB**: please refer to the scVI-dev notebook for introduction of the scVI package.

In this notebook, we investigate how semi-supervised learning combined with the probabilistic modelling of latent variables in scVI can help address the annotation problem.

The annotation problem consists in labelling cells, ie. **inferring their cell types**, knowing only a part of the labels.

In [1]:
cd ../..

/home/jeff/git/scVI


In [2]:
from scvi.dataset import CortexDataset
from scvi.models import SVAEC, VAE
from scvi.inference import JointSemiSupervisedVariationalInference

We instantiate the SVAEC model and train it over 250 epochs. Only labels from the `data_loader_labelled` will be used, but to cross validate the results, the labels of `data_loader_unlabelled` will is used at test time. The accuracy of the `unlabelled` dataset reaches 93% here at the end of training.

In [3]:
gene_dataset = CortexDataset()

use_batches=False
use_cuda=True

svaec = SVAEC(gene_dataset.nb_genes, gene_dataset.n_batches, gene_dataset.n_labels)
infer = JointSemiSupervisedVariationalInference(svaec, gene_dataset, n_labelled_samples_per_class=10)
infer.train(n_epochs=50)

infer.accuracy('unlabelled')

File data/expression.bin already downloaded
Preprocessing Cortex data
Finished preprocessing Cortex data
training: 100%|██████████| 50/50 [00:12<00:00,  3.96it/s]


0.917206132879046

**Benchmarking against other algorithms**

We can compare ourselves against the random forest and SVM algorithms, where we do grid search with 3-fold cross validation to find the best hyperparameters of these algorithms. This is automatically performed through the functions **`compute_accuracy_svc`** and **`compute_accuracy_rf`**.

These functions should be given as input the numpy array corresponding to the equivalent dataloaders, which is the purpose of the **`get_raw_data`** method from `scvi.dataset.utils`.

The format of the result is an Accuracy named tuple object giving higher granularity information about the accuracy ie, with attributes:

- **unweighted**: the standard definition of accuracy

- **weighted**: we might give the same weight to all classes in the final accuracy results. Informative only if the dataset is unbalanced.

- **worst**: the worst accuracy score for the classes

- **accuracy_classes** : give the detail of the accuracy per classes


Compute the accuracy score for rf and svc

In [4]:
svc_scores, rf_scores = infer.svc_rf()
print("\nSVC score test :\n", svc_scores[1])
print("\nRF score train :\n", rf_scores[1])


SVC score test :
 Accuracy(unweighted=0.8701873935264054, weighted=0.8465908861701248, worst=0.7223650385604113, accuracy_classes=[0.794392523364486, 0.9066666666666666, 0.8857142857142857, 0.7954545454545454, 0.9345679012345679, 0.8869752421959096, 0.7223650385604113])

RF score train :
 Accuracy(unweighted=0.9281090289608177, weighted=0.9166938151705352, worst=0.8319672131147541, accuracy_classes=[0.958974358974359, 0.8319672131147541, 0.9169435215946844, 0.9090909090909091, 0.9810844892812106, 0.9173469387755102, 0.9014492753623189])
